# GENERAL VIEW OF THE DATA

## Packages

Installing packages

In [45]:
# !pip3 install numpy
# !pip3 install --upgrade pip
# !pip3 install pandas
# !pip3 install matplotlib

Importing packages

In [46]:
import time
import random
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from IPython.display import clear_output

Matplotlib is building the font cache; this may take a moment.


## Downloading data

Importing data

In [2]:
pick_data = pd.read_csv("/Users/ANDRES/Documents/GitHub/OBETAWarehousingAnalyticsProject/Data/003 pick_data.csv", header=None)

/var/folders/0m/9hrzmrmx5rvbnxjrsct8cd500000gn/T/ipykernel_11899/586391051.py:1: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  pick_data = pd.read_csv("/Users/ANDRES/Documents/GitHub/OBETAWarehousingAnalyticsProject/Data/003 pick_data.csv", header=None)


Head of data

In [3]:
print(pick_data.head())

   0    1   2        3  4   5   6                            7
0  2  SHL  48  7055448  1  29  St  2017-06-30 11:15:24.0000000
1  2  SHL  48  7055448  1  30  St  2017-06-30 11:22:35.0000000
2  2  SHL  48  7055448  1  30  St  2017-06-30 12:04:50.0000000
3  2  SHL  48  7055448  1  20  St  2017-06-30 12:04:51.0000000
4  2  SHL  48  7055448  1  30  St  2017-06-30 12:05:02.0000000


Renaming columns to match schema

In [4]:
pick_data.columns = ["product_id", "category", "source_order_id", "order_number", "position_in_order", "pick_volume", "quantity_unit", "date_time"]

In [5]:
pick_data.head()

,product_id,category,source_order_id,order_number,position_in_order,pick_volume,quantity_unit,date_time
0,2,SHL,48,7055448,1,29,St,2017-06-30 11:15:24.0000000
1,2,SHL,48,7055448,1,30,St,2017-06-30 11:22:35.0000000
2,2,SHL,48,7055448,1,30,St,2017-06-30 12:04:50.0000000
3,2,SHL,48,7055448,1,20,St,2017-06-30 12:04:51.0000000
4,2,SHL,48,7055448,1,30,St,2017-06-30 12:05:02.0000000


Matching data types

In [24]:
pick_data["product_id"] = pick_data["product_id"].astype(str)
pick_data["category"] = pick_data["category"].astype(str)
pick_data["source_order_id"] = pick_data["source_order_id"].astype(str)
pick_data["order_number"] = pick_data["order_number"].astype(str)
pick_data["position_in_order"] = pick_data["position_in_order"].astype(int)
pick_data["pick_volume"] = pick_data["pick_volume"].astype(int)
pick_data["quantity_unit"] = pick_data["quantity_unit"].astype(str)
pick_data["date_time"] = pd.to_datetime(pick_data["date_time"])


In [30]:
pick_data.head()

,product_id,category,source_order_id,order_number,position_in_order,pick_volume,quantity_unit,date_time
0,2,SHL,48,7055448,1,29,St,2017-06-30 11:15:24
1,2,SHL,48,7055448,1,30,St,2017-06-30 11:22:35
2,2,SHL,48,7055448,1,30,St,2017-06-30 12:04:50
3,2,SHL,48,7055448,1,20,St,2017-06-30 12:04:51
4,2,SHL,48,7055448,1,30,St,2017-06-30 12:05:02


Checking data types

In [26]:
pick_data.dtypes

product_id                   object
category                     object
source_order_id              object
order_number                 object
position_in_order             int64
pick_volume                   int64
quantity_unit                object
date_time            datetime64[ns]
dtype: object

Creating new date and time column

In [ ]:
pick_data["date"] = pick_data["date_time"].dt.date
pick_data["time"] = pick_data["date_time"].dt.hour

In [56]:
pick_data.head()

,product_id,category,source_order_id,order_number,position_in_order,pick_volume,quantity_unit,date_time,date,time
0,2,SHL,48,7055448,1,29,St,2017-06-30 11:15:24,2017-06-30,11
1,2,SHL,48,7055448,1,30,St,2017-06-30 11:22:35,2017-06-30,11
2,2,SHL,48,7055448,1,30,St,2017-06-30 12:04:50,2017-06-30,12
3,2,SHL,48,7055448,1,20,St,2017-06-30 12:04:51,2017-06-30,12
4,2,SHL,48,7055448,1,30,St,2017-06-30 12:05:02,2017-06-30,12


## Data description

Matching data types

In [58]:
pick_data["date"] = pd.to_datetime(pick_data["date_time"])

General information

In [59]:
pick_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33888990 entries, 0 to 33888989
Data columns (total 10 columns):
 #   Column             Dtype         
---  ------             -----         
 0   product_id         object        
 1   category           object        
 2   source_order_id    object        
 3   order_number       object        
 4   position_in_order  int64         
 5   pick_volume        int64         
 6   quantity_unit      object        
 7   date_time          datetime64[ns]
 8   date               datetime64[ns]
 9   time               int32         
dtypes: datetime64[ns](2), int32(1), int64(2), object(5)
memory usage: 2.4+ GB


Describing data

In [61]:
number_of_products = len(set(pick_data.product_id.values))
number_of_categories = len(set(pick_data.category.values))
number_of_source_order_ids = len(set(pick_data.source_order_id.values))
number_of_order_numbers = len(set(pick_data.order_number.values))
number_of_positions_in_order = len(set(pick_data.position_in_order.values))

print("The number of unique products is: ",  number_of_products)
print("The number of unique categories is: ", number_of_categories)
print("The number of unique source order ids is: ", number_of_source_order_ids)
print("The number of unique order numbers is: ", number_of_order_numbers)
print("The number of unique positions in order is: ", number_of_positions_in_order)
print("")
print("The first date_time of the picks_data is: ", pick_data.date_time.min())
print("The last date_time of the picks_data is: ", pick_data.date_time.max())
print("The first time of the picks_data is: ", pick_data.time.min())
print("The last time of the picks_data is: ", pick_data.time.max())

The number of unique products is:  97338
The number of unique categories is:  5
The number of unique source order ids is:  2
The number of unique order numbers is:  6956136
The number of unique positions in order is:  340

The first date_time of the picks_data is:  2011-06-23 00:00:01
The last date_time of the picks_data is:  2020-07-14 11:42:01
The first time of the picks_data is:  0
The last time of the picks_data is:  23


Missing values

In [63]:
pick_data.isna().sum()

product_id           0
category             0
source_order_id      0
order_number         0
position_in_order    0
pick_volume          0
quantity_unit        0
date_time            0
date                 0
time                 0
dtype: int64

Group analysis

______

## Data manipulation